In [5]:
# name: Eklavya Gupta
# university: University of Petroleum and Energy Studies
# course: B.Tech CSE AI ML   # currently in 5th semester
# sapid: 500093960
# university mail id: 500093960@stu.upes.ac.in
# personal mail id: emessage.eg@gmail.com

import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_image
from torchvision.models import resnet18
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_name, label = self.data[idx]
        image = read_image(img_name)
        image = Image.fromarray(image.permute(1, 2, 0).numpy())
        if self.transform:
            image = self.transform(image)
        return image, label

# Load and preprocess the data
data = pd.read_csv('dataset.csv')
le = LabelEncoder()
data['label'] = le.fit_transform(data['label'])
data = data.values

train_data, val_data = train_test_split(data, test_size=0.2, random_state=10)

transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]) # these values are commonly used

train_dataset = CustomDataset(train_data, transform=transform)
val_dataset = CustomDataset(val_data, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)

model = resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Linear(512, 3),
    nn.LogSoftmax(dim=1)
)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Accuracy: {100 * correct / total}%")

    if (epoch + 1) % 3 == 0:
        checkpoint_filename = f"model_epoch_{epoch+1}.pth"
        torch.save(model.state_dict(), checkpoint_filename)
        print(f"Saved model checkpoint as {checkpoint_filename}")

print("Finished Training")

C:\Users\EKLAVYA\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\EKLAVYA\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/50, Loss: 0.9794911573762479, Accuracy: 50.0%
Epoch 2/50, Loss: 0.9886556599235189, Accuracy: 54.285714285714285%
Epoch 3/50, Loss: 0.9936110231539478, Accuracy: 62.857142857142854%
Saved model checkpoint as model_epoch_3.pth
Epoch 4/50, Loss: 0.825663577657247, Accuracy: 57.142857142857146%
Epoch 5/50, Loss: 0.7867155217606089, Accuracy: 72.85714285714286%
Epoch 6/50, Loss: 0.7832854574789172, Accuracy: 77.14285714285714%
Saved model checkpoint as model_epoch_6.pth
Epoch 7/50, Loss: 0.7716415810898163, Accuracy: 68.57142857142857%
Epoch 8/50, Loss: 0.6424370074131782, Accuracy: 70.0%
Epoch 9/50, Loss: 0.7659608879113111, Accuracy: 75.71428571428571%
Saved model checkpoint as model_epoch_9.pth
Epoch 10/50, Loss: 0.6337135531062233, Accuracy: 88.57142857142857%
Epoch 11/50, Loss: 0.5857303246140372, Accuracy: 88.57142857142857%
Epoch 12/50, Loss: 0.5478889159048381, Accuracy: 82.85714285714286%
Saved model checkpoint as model_epoch_12.pth
Epoch 13/50, Loss: 0.5716702963454999, A